In [1]:
"""

190 page 기준 8분 정도
휴일 때 대략 100~200 page
평일 때 대략 400~500 page 

한 페이지에 15 기사

"""

'\n\n190 page 기준 8분 정도\n휴일 때 대략 100~200 page\n평일 때 대략 400~500 page \n\n한 페이지에 15 기사\n\n'

In [2]:
from datetime import date
import pandas as pd
from tqdm import notebook
import requests
from bs4 import BeautifulSoup
import re

start_date = date(2022, 1, 1)
end_date = date(2022, 1, 2)


In [3]:
def create_soup(url):

    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
    return soup

In [130]:
def get_articles(year,month,day):

    
    date_data = pd.DataFrame()
    
    year = str(year)
    month = str(month).zfill(2)
    day = str(day).zfill(2)
    date = year+month+day
    
    
    # 마지막 페이지 번호 추출
    url = f"https://news.daum.net/breakingnews/society?page=10&regDate={date}"
    soup = create_soup(url)
    last_page = soup.find("em",attrs ={"class": "num_page"})
    last_page_num = int(re.sub(r'[^0-9]', '', last_page.text))

    
    
    for i in notebook.tqdm(range(1,last_page_num+1)):
        url = f"https://news.daum.net/breakingnews/society?page={i}&regDate={date}"
        soup = create_soup(url)

        
    
        # 뉴스 리스트가 포함된 속성 추출
        news_list_part = soup.find('ul', attrs={'class':'list_news2 list_allnews'})
        
        # COLUME : press
        press_list = []
        press = news_list_part.find_all('span', attrs={'class':'info_news'})
        for _span in press:
            press_list.append(_span.text.split()[0])
        
        # 뉴스 링크 리스트 추출
        # COLUME : link
        link_list = []
        news_link_list = news_list_part.find_all('a',attrs={"class":"link_thumb"})
        for news_link in news_link_list:
            link_list.append(news_link["href"])
        
        
        
        # COLUME : articleid
        articleid = []
        for link in link_list:
            articleid.append(link.split("/")[-1])
        
        
        
        # 뉴스 링크 리스트 추출
        # COLUME : link
        news_link_list = news_list_part.find_all('a')

        # 링크마다 요청
        # COLUME : title, content ,journalist
        title_list = []
        journalist_list = []
        content_list = []

        for idx, link in enumerate(link_list):
            url = link
            article_soup = create_soup(url)
            article_soup = article_soup.find("article")
            
            ### title
            title = article_soup.find("h3", attrs={'class':"tit_view"})
            title_list.append(title.text)
            
            ### journalist
            journalist = article_soup.find("span", attrs={'class':"txt_info"})
            journalist_list.append(journalist.text)
            
            ### content
            content = article_soup.find("div", attrs={'class':"article_view"})
            
            # 이미지 제거 
            if content.select_one("p.link_figure") != None:
                content.select_one("p.link_figure").decompose()
                
            
            news_contents = content.find_all("p")
            news_content= ""
            for _p in news_contents:
                tmp = _p.text
                tmp = tmp.replace("\xa0","")
                tmp = tmp.replace("\\","")
                tmp = tmp.replace(journalist_list[idx],"")
                if "@" in tmp \
                or "[카카오톡]" in tmp \
                or "[전화]" in tmp \
                or "[메일]" in tmp \
                or "[온라인 제보]" in tmp \
                or "[저작권자(c) YTN & YTN plus 무단전재 및 재배포 금지]" in tmp \
                or "※ '당신의 제보가 뉴스가 됩니다'" in tmp :
                    continue
                        
                news_content += tmp

                

            content_list.append(news_content)
            
                
                
        data =  pd.DataFrame(zip(articleid, title_list,journalist_list,press_list,content_list, link_list))
            
        date_data = pd.concat([date_data, data])
    
    return date_data

In [131]:
# [카카오톡] YTN을 검색해 채널 추가 [전화] 02-398-8585 [메일] social@ytn.co.kr [온라인 제보] www.ytn.co.kr

In [132]:
# from dateutil.rrule import rrule, DAILY

# df = pd.DataFrame()

# for date in notebook.tqdm(list(rrule(DAILY, dtstart=start_date, until=end_date))):
#     df = pd.concat([df, get_articles(date.year, date.month, date.day)])
# df.reset_index(drop=True, inplace=True)

# df = get_articles(date.year, date.month, date.day)

In [133]:
df = get_articles(2022, 1, 1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [134]:
"""
HTTPSConnectionPool(host='v.daum.net', port=443): Max retries exceeded with url: /v/20220101121016727 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020C9A0B23A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))
"""


"\nHTTPSConnectionPool(host='v.daum.net', port=443): Max retries exceeded with url: /v/20220101121016727 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000020C9A0B23A0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))\n"

In [138]:
df.iloc[101][4]

' [이데일리  기자] 골판지 제조공장에서 40대 노동자가 끼임 사고로 사망하는 사고가 발생했다. A씨는 머리 등을 크게 다쳐 병원으로 옮겨졌으나 끝내 숨졌다.A씨는 기계 사이에 골판지가 걸려 이를 빼내려고 하다가 사고를 당한 것으로 알려졌다. 경찰은 기계 오작동 여부 등 사고 경위와 작업장 내 안전 수칙 준수 여부 등을 조사할 예정이다.'

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 12
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       141 non-null    object
 1   1       141 non-null    object
 2   2       141 non-null    object
 3   3       141 non-null    object
 4   4       141 non-null    object
 5   5       141 non-null    object
dtypes: object(6)
memory usage: 7.7+ KB
